In [5]:
import librosa
import matplotlib.pyplot as plt
import IPython.display as dis
import numpy as np
import os, sys
from tqdm import tqdm
import pandas as pd
import librosa.display

In [6]:
rawdata = pd.read_csv('Raw_data.csv')

In [7]:
lst = os.listdir('../melspectrogram_data')
data_x = []
data_y = []
for i in tqdm(range(len(rawdata))):
   data_x.append(np.load('../melspectrogram_data/' + str(rawdata['song_id'].iloc[i])+'.npy'))
   data_y.append(rawdata['senti_tag'].iloc[i])

100%|██████████| 1337/1337 [00:08<00:00, 150.47it/s]


In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(data_y)

In [11]:
wav_to_img_data = []
target = []

# 128*128 사이즈로 멜스펙토그램 이미지를 잘라서 담아두고
# 타겟값을 같이 추가한 넘파이배열을 만든다
for idx in tqdm(range(len(data_x))):
    for i in range(15):
        wav_to_img_data.append(data_x[idx][:, 128* i : 128 * (i+1)].reshape(128,128,-1))
        target.append(data_y[idx])
wav_to_img_data = np.array(wav_to_img_data)
target = np.array(target)

100%|██████████| 1337/1337 [00:00<00:00, 63843.80it/s]


In [29]:
import vgg
import tensorflow as tf

In [30]:
model = vgg.VGG_model()

In [31]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 128, 128, 64)      640       
_________________________________________________________________
batch_normalization_80 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
activation_80 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 128, 128, 64)      36928     
_________________________________________________________________
batch_normalization_81 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
activation_81 (Activation)   (None, 128, 128, 64)    

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',  metrics=['acc'])

In [15]:
history = model.fit(wav_to_img_data, target, epochs=1, batch_size=32, validation_split=0.2)

 32/502 [>.............................] - ETA: 1:08:32 - loss: 3.9327 - acc: 0.2646

KeyboardInterrupt: 